# Data ETL

Basic data extraction, transformation and loading into a SQLite database for further preprocessing.

- [Tweet object metadata](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)
- [User object metadata](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/user-object)

In [1]:
import os
from os.path import join
from sqlite3 import connect
import pandas as pd
import numpy as np

project_dir = join(os.getcwd(), os.pardir)
raw_dir = join(project_dir, 'data', 'raw')
interim_dir = join(project_dir, 'data', 'interim')

In [2]:
raw_fname = 'data_pull_sample.json'
df = pd.read_json(join(raw_dir, raw_fname), lines=True)

_____
### Exploring and Transforming the contents in the raw data

In [3]:
df.columns

Index(['created_at', 'id', 'id_str', 'text', 'display_text_range', 'source',
       'truncated', 'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'is_quote_status', 'quote_count', 'reply_count',
       'retweet_count', 'favorite_count', 'entities', 'favorited', 'retweeted',
       'filter_level', 'lang', 'matching_rules', 'retweeted_status',
       'extended_tweet', 'possibly_sensitive', 'extended_entities',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status',
       'quoted_status_permalink'],
      dtype='object')

In [4]:
df.isna().sum(0) / df.shape[0] # checks percentage of missing values in existing columns

created_at                   0.000000
id                           0.000000
id_str                       0.000000
text                         0.000000
display_text_range           0.875321
source                       0.000000
truncated                    0.000000
in_reply_to_status_id        0.933019
in_reply_to_status_id_str    0.933019
in_reply_to_user_id          0.923094
in_reply_to_user_id_str      0.923094
in_reply_to_screen_name      0.923132
user                         0.000000
geo                          0.999057
coordinates                  0.999057
place                        0.990340
contributors                 1.000000
is_quote_status              0.000000
quote_count                  0.000000
reply_count                  0.000000
retweet_count                0.000000
favorite_count               0.000000
entities                     0.000000
favorited                    0.000000
retweeted                    0.000000
filter_level                 0.000000
lang        

### Important information regarding each feature (Tweet objects): 
- "id" and "id_str" values differ, use "id_str"
- "text" does not contain the full text of the tweet (ends with "...")
- "source" can possibly be used to detect device type (android/ios/pc/mac/ipad...)
- "truncated" indicates whether the text parameter was truncated. It goes with the "extended_tweet" parameter
- "extended_tweet" contains the full tweet if "truncated" is True
- "in_reply_to_status_id" ignore. Use "in_reply_to_status_id_str" instead
- "in_reply_to_user_id" ignore. Use "in_reply_to_user_id_str" instead
- "user" contains information about the user posting the tweet. It is subdivided into many other fields (user-related)
- "coordinates" is nearly always empty. It should probably be disregarded.
- "place" is slightly more common than "coordinates", although still very rare. Probably should be disregarded.
- "quoted_status_id" ignore. Use "quoted_status_id_str" instead. Although this field can probably be dropped.
- "retweeted_status" indicates whether it's a retweet. Must be kept.
- "entities" contains info of Twitter's own text parsing
- "favorited" should be dropped
- "retweeted" should be dropped
- "filter_level" can be dropped
- "lang" indicates a BCP 47 language identifier corresponding to the machine-detected language of the Tweet text
- "matching_rules" should be dropped
- "geo" should be dropped. Use the "coordinates" field instead
- "retweeted_status", retweets can be distinguished from typical Tweets by the existence of a retweeted_status attribute. This attribute contains a representation of the original Tweet that was retweeted.

### Important information regarding each feature (User objects): 
- "id" and "id_str" values differ, use "id_str"
- "name" is the name of the user, as they’ve defined it. Not necessarily a person’s name.
- "screen_name" is the handle, or alias that this user identifies themselves with. screen_names are unique but subject to change. Use id_str as a user identifier whenever possible.
- "location" user-defined location of the user. Nearly 40% missing values.
- "derived" contains its own object. Collection of Enrichment metadata derived for user. Mostly related with user location. Inexistent in the sample dataset.
- "url" user's associated website. Can be dropped (76% missing values).
- "description" is the user-defined UTF-8 string describing their account. Can be used for feature extraction.
- "protected" when true, indicates that this user has chosen to protect their Tweets. Confirm all Users have False. Can dropped afterwards.
- "verified" indicates that the user has a verified account. 
- "friends_count" number of users this account is following.
- "listed_count" number of public lists that this user is a member of.
- "statuses_count" number of Tweets (including retweets) issued by the user.
- "default_profile" can be dropped.
- "profile_background_tile" can be dropped. (deprecated)
- Many features contain 100% missing values, mostly due to feature deprecation. Should be dropped.

In [5]:
df_users = pd.DataFrame(df['user'].tolist())

# I am deleting "entities" and "extended_entities" because I was unsure
# of its advantage. Maybe it can be properly preprocessed, or at maybe
# we can just take the useful bits out of it.
df_tweets = df.drop(columns=[
    'id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'user',
    'coordinates', 'place', 'quoted_status_id', 'favorited',
    'retweeted', 'retweeted_status', 'matching_rules', 'geo', 
    'filter_level', 'display_text_range', 'contributors',
    'quoted_status', 'quoted_status_id', 'quoted_status_permalink',
    'in_reply_to_screen_name', 'text', 'extended_tweet', 'truncated',
    'entities', 'extended_entities'
])
df_tweets['user_id_str'] = df['user'].apply(lambda x: x['id_str'])
df_tweets['full_text'] = df.apply(
    lambda row: 
        row['text'] 
        if not row['truncated'] 
        else row['extended_tweet']['full_text'], 
    axis=1
)

def get_retweet_id(row):
    """returns: is_retweet, original_tweet_id_str"""
    if type(row['retweeted_status']) == dict:
        return True, row['retweeted_status']['id_str']
    else:
        return False, np.nan

df_tweets['is_retweet'], df_tweets['original_tweet_id_str'] = zip(*df.apply(get_retweet_id, axis=1))
df_tweets['is_reply'] = ~df['in_reply_to_status_id'].isna()

In [6]:
df_users.isna().sum(0) / df_users.shape[0]

id                                    0.000000
id_str                                0.000000
name                                  0.000000
screen_name                           0.000000
location                              0.368377
url                                   0.759811
description                           0.190151
translator_type                       0.000000
derived                               0.498566
protected                             0.000000
verified                              0.000000
followers_count                       0.000000
friends_count                         0.000000
listed_count                          0.000000
favourites_count                      0.000000
statuses_count                        0.000000
created_at                            0.000000
utc_offset                            1.000000
time_zone                             1.000000
geo_enabled                           0.000000
lang                                  1.000000
contributors_

In [7]:
# drop columns with 100% missing values
all_missing = df_users.columns[(df_users.isna().sum(0) / df_users.shape[0]) == 1].tolist()

df_users = df_users.drop(columns=[
    'id', 'url', 'default_profile', 'profile_image_url', 
    'profile_image_url_https', 'profile_banner_url', 'profile_background_image_url',
    'profile_background_image_url_https', 'profile_background_tile', 'profile_link_color',
    'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image',
    'default_profile_image', 'translator_type', 'contributors_enabled', 'is_translator',
    'profile_background_color', 'profile_sidebar_border_color'
]+all_missing).drop_duplicates(subset='id_str', keep='first')

df_users['derived__location'] = df_users['derived']\
    .apply(
        lambda x: x['locations'][0]['country'] 
        if type(x)==dict 
        else x
    )

df_users['derived'] = df_users['derived'].apply(lambda x: str(x) if type(x)==dict else x)

In [8]:
print('Sanity check: dropping duplicate rows VS dropping duplicate user IDs')
print(df_users.drop_duplicates().shape)
print(df_users.drop_duplicates(subset='id_str').shape)

# After some research, I found this inconsistency comes from the features 'favourites_count' and 'statuses_count'
# Two users probably had different figures when (re)tweeting posts that match the original search query

Sanity check: dropping duplicate rows VS dropping duplicate user IDs
(11358, 16)
(11358, 16)


_____
### Visualizing transformed data

In [9]:
df_users.head()

,id_str,name,screen_name,location,description,derived,protected,verified,followers_count,friends_count,listed_count,favourites_count,statuses_count,created_at,geo_enabled,derived__location
0,1065957356079476736,大天使糖君…。❤️💢,torured13,北赤羽,(北)赤羽の妖精‼️😫\n遂に帰宅はしたけれど‼️🤔😫🏠👣\nADHDに適応障害に自閉症スペ...,{'locations': [{'country': 'Equatorial Guinea'...,False,False,163,343,2,44864,28739,Fri Nov 23 13:16:53 +0000 2018,True,Equatorial Guinea
2,901828520,JillurRahaman(Bapi),Bapigharami,"Hasnabad, Taki","Secretary,District YouthCongressN24PGS(Rural)....","{'locations': [{'country': 'India', 'country_c...",False,False,41,359,0,2041,1894,Wed Oct 24 13:28:37 +0000 2012,False,India
3,921596113949417472,NORTH 24 PGS DYC(Rural),GharamiBapi,"Basirhat,Hasnabad, Taki","Secretary,District youth congress N24PGS.Ex Pr...",NaN,False,False,48,277,0,2348,2050,Sat Oct 21 04:36:50 +0000 2017,False,NaN
4,906562306401755136,ぴ³＠パチンコ自粛ニキʚ🎧ɞ,nipaaaaaa113,天照大御神,どもぉ〜。ぴぴぴと申しますぅぅぅぅ( *°∀°* )アニオタデェェェェェェェェスッ!!⭐️🌟...,NaN,False,False,4051,4304,24,17644,12746,Sat Sep 09 16:57:51 +0000 2017,True,NaN
6,1256085380106842113,Thiên Nhi,ThienNhi0211,None,Waanjai MewGulf ❤❤❤\nHậu Cung MewGulf ❤Waanjai...,NaN,False,False,89,627,0,1288,11304,Fri May 01 04:58:01 +0000 2020,False,NaN


In [10]:
df_tweets.head()

,created_at,id_str,source,in_reply_to_status_id_str,in_reply_to_user_id_str,is_quote_status,quote_count,reply_count,retweet_count,favorite_count,lang,possibly_sensitive,quoted_status_id_str,user_id_str,full_text,is_retweet,original_tweet_id_str,is_reply
0,2020-06-14 23:57:21+00:00,1272317232626888704,"<a href=""http://twitter.com/download/android"" ...",1.272317e+18,4.844328e+09,False,0,1,0,0,ja,NaN,NaN,1065957356079476736,@Ampan_C そのコメント欄に必ず私が居た‼️🤔🙄🙄🙄\n自分で質問して自分が最初にコメ...,False,NaN,True
1,2020-06-14 23:51:31+00:00,1272315765975183360,"<a href=""http://twitter.com/download/android"" ...",1.272312e+18,4.844328e+09,False,0,1,0,0,ja,NaN,NaN,1065957356079476736,@Ampan_C X軸とY軸が有るグラフで虚数を掛けると90度回転する‼️🤔,False,NaN,True
2,2020-06-14 23:51:02+00:00,1272315644495581184,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,False,0,0,0,0,en,NaN,NaN,901828520,RT @IYCWestBengal: Today #WBPYC President @Sha...,True,1272197734913544193,False
3,2020-06-14 23:50:39+00:00,1272315545027637248,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,False,0,0,0,0,en,NaN,NaN,921596113949417472,RT @IYCWestBengal: Today #WBPYC President @Sha...,True,1272197734913544193,False
4,2020-06-14 23:49:53+00:00,1272315355700981760,"<a href=""http://twitter.com/download/iphone"" r...",1.272315e+18,4.844328e+09,False,0,1,0,0,ja,NaN,NaN,906562306401755136,@Ampan_C あんぱんおぱよー！,False,NaN,True


_____
### Loading data into database

In [11]:

with connect(join(interim_dir, raw_fname.replace('.json', '.db'))) as connection:
    df_users.to_sql('users', connection, index=False, if_exists='replace')
    df_tweets.to_sql('tweets', connection, index=False, if_exists='replace')
